# Day 04: Backtesting Framework

## Week 24 - Capstone Project

### Learning Objectives
- Build a production-grade backtesting framework
- Implement realistic transaction costs and slippage
- Apply walk-forward optimization
- Calculate comprehensive risk metrics
- Avoid common backtesting pitfalls (look-ahead bias, survivorship bias)

### Why Backtesting Matters
- **Validate strategies** before risking real capital
- **Understand risk characteristics** (drawdowns, volatility)
- **Estimate realistic performance** with transaction costs
- **Detect overfitting** with out-of-sample testing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum

np.random.seed(42)
print("✅ Libraries loaded successfully!")

## 1. Backtesting Framework Classes

In [ ]:
class Signal(Enum):
    """Trading signals."""
    STRONG_BUY = 2
    BUY = 1
    HOLD = 0
    SELL = -1
    STRONG_SELL = -2


@dataclass
class BacktestConfig:
    """Configuration for backtesting."""
    initial_capital: float = 100000.0
    commission_rate: float = 0.001  # 0.1% per trade
    slippage_rate: float = 0.0005   # 0.05% slippage
    max_position_size: float = 0.20  # Max 20% per position
    stop_loss_pct: float = 0.05     # 5% stop loss
    take_profit_pct: float = 0.10   # 10% take profit
    risk_free_rate: float = 0.05    # 5% annual risk-free rate


@dataclass
class Trade:
    """Record of a single trade."""
    ticker: str
    entry_date: datetime
    entry_price: float
    shares: int
    direction: int  # 1 for long, -1 for short
    exit_date: Optional[datetime] = None
    exit_price: Optional[float] = None
    pnl: Optional[float] = None
    return_pct: Optional[float] = None


@dataclass
class BacktestResults:
    """Results from backtesting."""
    # Performance Metrics
    total_return: float
    annual_return: float
    volatility: float
    sharpe_ratio: float
    sortino_ratio: float
    max_drawdown: float
    calmar_ratio: float
    
    # Trading Metrics
    num_trades: int
    win_rate: float
    profit_factor: float
    avg_trade_return: float
    
    # Time Series
    equity_curve: pd.Series
    daily_returns: pd.Series
    trades: List[Trade]
    
    def summary(self) -> str:
        """Generate summary report."""
        return f"""
{'='*60}
                  BACKTEST RESULTS SUMMARY
{'='*60}

📊 PERFORMANCE METRICS
{'-'*40}
Total Return:       {self.total_return*100:>10.2f}%
Annual Return:      {self.annual_return*100:>10.2f}%
Volatility:         {self.volatility*100:>10.2f}%
Sharpe Ratio:       {self.sharpe_ratio:>10.2f}
Sortino Ratio:      {self.sortino_ratio:>10.2f}
Max Drawdown:       {self.max_drawdown*100:>10.2f}%
Calmar Ratio:       {self.calmar_ratio:>10.2f}

📈 TRADING METRICS
{'-'*40}
Number of Trades:   {self.num_trades:>10}
Win Rate:           {self.win_rate*100:>10.2f}%
Profit Factor:      {self.profit_factor:>10.2f}
Avg Trade Return:   {self.avg_trade_return*100:>10.2f}%

{'='*60}
"""

print("✅ Backtest classes defined!")

## 2. Backtester Implementation

In [ ]:
class Backtester:
    """
    Production-grade backtester for trading strategies.
    
    Features:
    - Transaction costs and slippage
    - Position sizing
    - Risk management (stop loss, take profit)
    - Comprehensive metrics
    """
    
    def __init__(self, config: BacktestConfig = None):
        self.config = config or BacktestConfig()
        self.trades: List[Trade] = []
        self.positions: Dict[str, Trade] = {}
    
    def run(self, prices: pd.DataFrame, signals: pd.DataFrame) -> BacktestResults:
        """
        Run backtest on given prices and signals.
        
        Args:
            prices: DataFrame of prices (dates x tickers)
            signals: DataFrame of signals (dates x tickers), values -2 to 2
        
        Returns:
            BacktestResults with performance metrics
        """
        self.trades = []
        self.positions = {}
        
        capital = self.config.initial_capital
        cash = capital
        equity_values = []
        
        for date in prices.index:
            # Check stop loss / take profit for existing positions
            self._check_exits(date, prices.loc[date])
            
            # Process new signals
            if date in signals.index:
                for ticker in signals.columns:
                    if ticker in prices.columns:
                        signal = signals.loc[date, ticker]
                        price = prices.loc[date, ticker]
                        
                        # Process signal
                        cash = self._process_signal(date, ticker, signal, price, cash, capital)
            
            # Calculate equity
            position_value = sum(
                pos.shares * prices.loc[date, pos.ticker] * pos.direction
                for pos in self.positions.values()
                if pos.ticker in prices.columns
            )
            equity = cash + position_value
            equity_values.append({'date': date, 'equity': equity})
        
        # Close remaining positions
        final_date = prices.index[-1]
        for ticker in list(self.positions.keys()):
            self._close_position(ticker, final_date, prices.loc[final_date, ticker])
        
        # Calculate results
        equity_curve = pd.DataFrame(equity_values).set_index('date')['equity']
        
        return self._calculate_results(equity_curve)
    
    def _process_signal(self, date, ticker, signal, price, cash, capital) -> float:
        """Process a trading signal."""
        # Skip if no clear signal
        if abs(signal) < 1:
            return cash
        
        direction = 1 if signal > 0 else -1
        
        # Check if already in position
        if ticker in self.positions:
            existing = self.positions[ticker]
            # Close if signal direction changed
            if existing.direction != direction:
                cash += self._close_position(ticker, date, price)
            else:
                return cash  # Already in position
        
        # Calculate position size
        max_value = capital * self.config.max_position_size
        position_value = min(max_value, cash * 0.9)  # Leave some cash buffer
        
        # Apply transaction costs
        effective_price = price * (1 + self.config.slippage_rate * direction)
        shares = int(position_value / effective_price)
        
        if shares <= 0:
            return cash
        
        cost = shares * effective_price
        commission = cost * self.config.commission_rate
        
        if cost + commission > cash:
            return cash
        
        # Open position
        trade = Trade(
            ticker=ticker,
            entry_date=date,
            entry_price=effective_price,
            shares=shares,
            direction=direction
        )
        self.positions[ticker] = trade
        
        return cash - cost - commission
    
    def _close_position(self, ticker: str, date, price: float) -> float:
        """Close a position and return cash value."""
        if ticker not in self.positions:
            return 0
        
        trade = self.positions[ticker]
        
        # Apply slippage
        effective_price = price * (1 - self.config.slippage_rate * trade.direction)
        
        # Calculate P&L
        gross_pnl = trade.shares * (effective_price - trade.entry_price) * trade.direction
        commission = trade.shares * effective_price * self.config.commission_rate
        net_pnl = gross_pnl - commission
        
        # Update trade record
        trade.exit_date = date
        trade.exit_price = effective_price
        trade.pnl = net_pnl
        trade.return_pct = (effective_price / trade.entry_price - 1) * trade.direction
        
        self.trades.append(trade)
        del self.positions[ticker]
        
        return trade.shares * effective_price - commission
    
    def _check_exits(self, date, prices: pd.Series):
        """Check stop loss and take profit."""
        for ticker in list(self.positions.keys()):
            if ticker not in prices.index:
                continue
            
            trade = self.positions[ticker]
            current_price = prices[ticker]
            
            # Calculate return
            current_return = (current_price / trade.entry_price - 1) * trade.direction
            
            # Check stop loss
            if current_return < -self.config.stop_loss_pct:
                self._close_position(ticker, date, current_price)
            # Check take profit
            elif current_return > self.config.take_profit_pct:
                self._close_position(ticker, date, current_price)
    
    def _calculate_results(self, equity_curve: pd.Series) -> BacktestResults:
        """Calculate comprehensive results."""
        # Returns
        daily_returns = equity_curve.pct_change().dropna()
        
        # Performance metrics
        total_return = equity_curve.iloc[-1] / equity_curve.iloc[0] - 1
        n_days = len(equity_curve)
        annual_return = (1 + total_return) ** (252 / n_days) - 1
        volatility = daily_returns.std() * np.sqrt(252)
        
        # Sharpe ratio
        excess_return = annual_return - self.config.risk_free_rate
        sharpe_ratio = excess_return / volatility if volatility > 0 else 0
        
        # Sortino ratio
        downside_returns = daily_returns[daily_returns < 0]
        downside_vol = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
        sortino_ratio = excess_return / downside_vol if downside_vol > 0 else 0
        
        # Max drawdown
        cummax = equity_curve.cummax()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown = abs(drawdown.min())
        
        # Calmar ratio
        calmar_ratio = annual_return / max_drawdown if max_drawdown > 0 else 0
        
        # Trading metrics
        winning_trades = [t for t in self.trades if t.pnl and t.pnl > 0]
        losing_trades = [t for t in self.trades if t.pnl and t.pnl <= 0]
        
        num_trades = len(self.trades)
        win_rate = len(winning_trades) / num_trades if num_trades > 0 else 0
        
        gross_profit = sum(t.pnl for t in winning_trades) if winning_trades else 0
        gross_loss = abs(sum(t.pnl for t in losing_trades)) if losing_trades else 1
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0
        
        avg_trade_return = np.mean([t.return_pct for t in self.trades if t.return_pct]) if self.trades else 0
        
        return BacktestResults(
            total_return=total_return,
            annual_return=annual_return,
            volatility=volatility,
            sharpe_ratio=sharpe_ratio,
            sortino_ratio=sortino_ratio,
            max_drawdown=max_drawdown,
            calmar_ratio=calmar_ratio,
            num_trades=num_trades,
            win_rate=win_rate,
            profit_factor=profit_factor,
            avg_trade_return=avg_trade_return,
            equity_curve=equity_curve,
            daily_returns=daily_returns,
            trades=self.trades
        )

print("✅ Backtester class implemented!")

## 3. Signal Generation Strategy

In [ ]:
# Download data
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
START_DATE = '2020-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"📥 Downloading data...")
data = yf.download(TICKERS, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
prices = data['Close'].dropna()
print(f"✅ Data: {prices.shape[0]} days, {prices.shape[1]} tickers")

In [ ]:
def generate_signals(prices: pd.DataFrame) -> pd.DataFrame:
    """
    Generate trading signals based on multiple indicators.
    
    Signal values:
        2: Strong Buy
        1: Buy
        0: Hold
       -1: Sell
       -2: Strong Sell
    """
    signals = pd.DataFrame(index=prices.index, columns=prices.columns, data=0)
    
    for ticker in prices.columns:
        price = prices[ticker]
        
        # Moving averages
        sma_20 = price.rolling(20).mean()
        sma_50 = price.rolling(50).mean()
        
        # Momentum
        momentum_10 = price / price.shift(10) - 1
        
        # RSI
        delta = price.diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        
        # Generate signals
        signal = pd.Series(0, index=prices.index)
        
        # Trend signal
        trend_signal = np.where(sma_20 > sma_50, 1, -1)
        
        # Momentum signal
        mom_signal = np.where(momentum_10 > 0.05, 1, np.where(momentum_10 < -0.05, -1, 0))
        
        # RSI signal
        rsi_signal = np.where(rsi < 30, 1, np.where(rsi > 70, -1, 0))
        
        # Combined signal
        combined = trend_signal + mom_signal + rsi_signal
        signal = np.clip(combined, -2, 2)
        
        signals[ticker] = signal
    
    return signals.dropna()

signals = generate_signals(prices)
print(f"\n📊 Signals generated: {signals.shape}")
print(f"\nSignal distribution:")
print(signals.apply(pd.Series.value_counts).fillna(0).astype(int))

## 4. Run Backtest

In [ ]:
# Configure and run backtest
config = BacktestConfig(
    initial_capital=100000,
    commission_rate=0.001,
    slippage_rate=0.0005,
    max_position_size=0.20,
    stop_loss_pct=0.05,
    take_profit_pct=0.10
)

backtester = Backtester(config)
results = backtester.run(prices, signals)

print(results.summary())

In [ ]:
# Plot equity curve
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Equity curve
axes[0].plot(results.equity_curve, linewidth=1.5, color='blue')
axes[0].axhline(y=config.initial_capital, color='gray', linestyle='--', alpha=0.5)
axes[0].set_title('Equity Curve', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Portfolio Value ($)')
axes[0].grid(True, alpha=0.3)

# Drawdown
cummax = results.equity_curve.cummax()
drawdown = (results.equity_curve - cummax) / cummax * 100
axes[1].fill_between(drawdown.index, drawdown, 0, color='red', alpha=0.3)
axes[1].set_title('Drawdown', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Drawdown (%)')
axes[1].grid(True, alpha=0.3)

# Daily returns
axes[2].bar(results.daily_returns.index, results.daily_returns * 100, 
            color=['green' if x > 0 else 'red' for x in results.daily_returns],
            width=1, alpha=0.6)
axes[2].set_title('Daily Returns', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Return (%)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Benchmark Comparison

In [ ]:
# Compare with Buy & Hold
benchmark = yf.download('SPY', start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)['Close']
benchmark_returns = benchmark.pct_change().dropna()
benchmark_equity = config.initial_capital * (1 + benchmark_returns).cumprod()

# Align indices
common_dates = results.equity_curve.index.intersection(benchmark_equity.index)
strategy_aligned = results.equity_curve.loc[common_dates]
benchmark_aligned = benchmark_equity.loc[common_dates]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(strategy_aligned, label='Strategy', linewidth=1.5)
ax.plot(benchmark_aligned, label='SPY Buy & Hold', linewidth=1.5, alpha=0.7)
ax.axhline(y=config.initial_capital, color='gray', linestyle='--', alpha=0.5)
ax.set_title('Strategy vs Benchmark (SPY)', fontsize=12, fontweight='bold')
ax.set_ylabel('Portfolio Value ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate benchmark metrics
bench_total_return = benchmark_aligned.iloc[-1] / benchmark_aligned.iloc[0] - 1
bench_annual_return = (1 + bench_total_return) ** (252 / len(benchmark_aligned)) - 1
bench_vol = benchmark_returns.std() * np.sqrt(252)
bench_sharpe = (bench_annual_return - config.risk_free_rate) / bench_vol

print("\n" + "="*60)
print("STRATEGY VS BENCHMARK COMPARISON")
print("="*60)
print(f"{'Metric':<25} {'Strategy':>15} {'SPY B&H':>15}")
print("-"*60)
print(f"{'Total Return':<25} {results.total_return*100:>14.2f}% {bench_total_return*100:>14.2f}%")
print(f"{'Annual Return':<25} {results.annual_return*100:>14.2f}% {bench_annual_return*100:>14.2f}%")
print(f"{'Sharpe Ratio':<25} {results.sharpe_ratio:>15.2f} {bench_sharpe:>15.2f}")
print(f"{'Max Drawdown':<25} {results.max_drawdown*100:>14.2f}% {'N/A':>15}")
print("="*60)

## 6. Key Takeaways

### What We Built:
1. **Production Backtester**: Transaction costs, slippage, position sizing
2. **Risk Management**: Stop loss, take profit
3. **Comprehensive Metrics**: Sharpe, Sortino, Max DD, Win Rate, Profit Factor
4. **Benchmark Comparison**: SPY Buy & Hold baseline

### Tomorrow: Risk Management Integration
- VaR and CVaR calculations
- Position sizing frameworks
- Portfolio risk decomposition